# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [9]:
cities_csv = "../python-api-challenge/output_data/cities.csv"
cities_weather_df = pd.read_csv(cities_csv)

cities_weather_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,busselton,-33.6500,115.3333,61.00,94,100,3.00,AU,1612671072
1,1,vaini,-21.2000,-175.2000,82.40,83,75,9.22,TO,1612671178
2,2,kjollefjord,70.9457,27.3465,25.84,94,100,22.41,NO,1612671179
3,3,najran,17.4924,44.1277,57.20,54,0,4.61,SA,1612670940
4,4,hilo,19.7297,-155.0900,75.20,57,90,6.91,US,1612671180


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [10]:
# configure api key
gmaps.configure(api_key=g_key)

# store cities
locations = cities_weather_df[["Lat", "Lng"]]

humidity = cities_weather_df["Humidity"].astype(float)

In [12]:
# Plot Heatmap
humid_map = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

# Add layer
humid_map.add_layer(heat_layer)

# Display figure
humid_map

Figure(layout=FigureLayout(height='420px'))

In [13]:
weather_dropna = cities_weather_df.dropna()

weather_dropna.head(20)

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,busselton,-33.6500,115.3333,61.00,94,100,3.00,AU,1612671072
1,1,vaini,-21.2000,-175.2000,82.40,83,75,9.22,TO,1612671178
2,2,kjollefjord,70.9457,27.3465,25.84,94,100,22.41,NO,1612671179
3,3,najran,17.4924,44.1277,57.20,54,0,4.61,SA,1612670940
4,4,hilo,19.7297,-155.0900,75.20,57,90,6.91,US,1612671180
5,5,richards bay,46.8361,-95.9123,-16.60,69,75,6.91,US,1612671181
6,6,kavieng,-2.5744,150.7967,85.14,63,35,3.94,PG,1612671181
7,7,alofi,-19.0595,-169.9187,84.20,79,100,10.36,NU,1612671182
8,8,tasiilaq,65.6145,-37.6368,32.00,59,20,20.71,GL,1612671182
9,9,pochutla,15.7432,-96.4661,78.01,82,4,1.01,MX,1612671182


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
# A max temperature lower than 80 degrees but higher than 70.
cities_over70 = weather_dropna[weather_dropna["Max Temp"] > 70]
cities_under80 = cities_over70[cities_over70["Max Temp"] < 80]

# Wind speed less than 10 mph.
cities_less10wspeed = cities_under80[cities_under80["Wind Speed"] < 10]

# Zero cloudiness.
ideal_weather = cities_less10wspeed[cities_less10wspeed["Cloudiness"] == 0]



ideal_weather

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,11,young,-32.6833,-57.6333,71.01,77,0,4.65,UY,1612671183
115,115,manzanillo,20.3433,-77.1167,74.21,82,0,5.70,CU,1612671225
132,132,victoria,22.2855,114.1577,73.00,62,0,4.00,HK,1612671195
226,226,methven,-43.6333,171.6500,72.00,59,0,3.00,NZ,1612671263
269,269,hambantota,6.1241,81.1185,78.80,94,0,8.05,LK,1612671282
307,307,barroquinha,-3.0189,-41.1361,73.45,90,0,4.21,BR,1612671297
442,442,lazaro cardenas,19.4558,-71.3261,73.00,82,0,5.01,DO,1612671354
446,446,moron,-34.6534,-58.6198,73.99,91,0,3.00,AR,1612671356
496,496,tecoanapa,16.5167,-98.7500,75.74,77,0,4.99,MX,1612671371


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [19]:
hotel_map = ideal_weather[["Lat", "Lng"]]

hotel_humid = ideal_weather["Humidity"].astype(float)

# Plot Heatmap
hotel_plot = gmaps.figure()

# Create heat layer
hotel_heat_layer = gmaps.heatmap_layer(hotel_map, weights=hotel_humid, 
                                 dissipating=False, max_intensity=50,
                                 point_radius=2.5)

# Add layer
hotel_plot.add_layer(hotel_heat_layer)

# Display figure
hotel_plot

Figure(layout=FigureLayout(height='420px'))

In [30]:
hotels = []

# Loop through narrowed down dataframe to get nearest hotel
for city in range(len(ideal_weather["City"])):

    lat = ideal_weather.iloc[city]["Lat"]
    lng = ideal_weather.iloc[city]["Lng"]

    city_location = f"{lat},{lng}"

    params = {
        "location": city_location, 
        "types": "lodging",
        "radius": 5000,
        "key": g_key
    }

    query_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    hotel_request = requests.get(query_url, params=params)
    hotel_response = hotel_request.json()

    try:
        hotels.append(hotel_response["results"][0]["name"])
    except:
        hotels.append("Nearest hotel not found")

# Dataframe with nearest hotel
ideal_weather["Hotel Name"] = hotels
ideal_weather

<ipython-input-30-de2d02846831>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal_weather["Hotel Name"] = hotels


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Nearest Hotel,Hotel Name
11,11,young,-32.6833,-57.6333,71.01,77,0,4.65,UY,1612671183,Posada Gira el Sol,Posada Gira el Sol
115,115,manzanillo,20.3433,-77.1167,74.21,82,0,5.70,CU,1612671225,"HOSTAL ""THE YELLOW HOUSE""","HOSTAL ""THE YELLOW HOUSE"""
132,132,victoria,22.2855,114.1577,73.00,62,0,4.00,HK,1612671195,Mini Hotel Central,Mini Hotel Central
226,226,methven,-43.6333,171.6500,72.00,59,0,3.00,NZ,1612671263,Brinkley Resort,Brinkley Resort
269,269,hambantota,6.1241,81.1185,78.80,94,0,8.05,LK,1612671282,Bungalow 63,Bungalow 63
307,307,barroquinha,-3.0189,-41.1361,73.45,90,0,4.21,BR,1612671297,Pousada Fontenele,Pousada Fontenele
442,442,lazaro cardenas,19.4558,-71.3261,73.00,82,0,5.01,DO,1612671354,Hotel Marien,Hotel Marien
446,446,moron,-34.6534,-58.6198,73.99,91,0,3.00,AR,1612671356,Hotel Morón,Hotel Morón
496,496,tecoanapa,16.5167,-98.7500,75.74,77,0,4.99,MX,1612671371,Hotel Manglares,Hotel Manglares


In [31]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_weather.iterrows()]
locations = ideal_weather[["Lat", "Lng"]]

In [32]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content= [f"Hotel Name: {hotel}" for hotel in hotels])
hotel_plot.add_layer(markers)

# Display figure
hotel_plot

Figure(layout=FigureLayout(height='420px'))